In [12]:
import os
import sys
import glob
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [28]:
df_all = pd.read_csv('s3://stormpetrels/samples/labels/samples_all.csv')

ConnectTimeoutError: Connect timeout on endpoint URL: "https://stormpetrels.s3.eu-central-1.amazonaws.com/samples/labels/samples_all.csv"

In [19]:
nice_features =  ['freq_Q25', 'freq_Q75', 'freq_mean', 'freq_median',
       'freq_mode', 'freq_peak', 'offset', 'onset', 'pitch_IQR',
       'pitch_Q25', 'pitch_Q75', 'pitch_max', 'pitch_mean',
       'pitch_median', 'pitch_min', 'yaafe_LPC', 'yaafe_LSF.0', 'yaafe_LSF.1',
       'yaafe_LSF.2', 'yaafe_LSF.3', 'yaafe_LSF.4', 'yaafe_LSF.5',
       'yaafe_LSF.6', 'yaafe_LSF.7', 'yaafe_LSF.8', 'yaafe_LSF.9',
       'yaafe_MFCC.0', 'yaafe_MFCC.1', 'yaafe_MFCC.10', 'yaafe_MFCC.11',
       'yaafe_MFCC.12', 'yaafe_MFCC.2', 'yaafe_MFCC.3', 'yaafe_MFCC.4',
       'yaafe_MFCC.5', 'yaafe_MFCC.6', 'yaafe_MFCC.7', 'yaafe_MFCC.8',
       'yaafe_MFCC.9', 'yaafe_OBSI.0', 'yaafe_OBSI.1', 'yaafe_OBSI.2',
       'yaafe_OBSI.3', 'yaafe_OBSI.4', 'yaafe_OBSI.5', 'yaafe_OBSI.6',
       'yaafe_OBSI.7', 'yaafe_OBSI.8', 'yaafe_SpectralFlatness',
       'yaafe_SpectralFlux', 'yaafe_SpectralRolloff',
       'yaafe_SpectralVariation']

In [5]:
df = df_all[nice_features]

In [6]:
y = df['petrel']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df.values, y, test_size=0.3, random_state=42)

In [8]:
dtrain = xgb.DMatrix(X_train, label=y_train.values)
dtest = xgb.DMatrix(X_test, label=y_test.values)

In [10]:
param = {'max_depth': 3, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic', 'subsample': 0.8, 'gamma': 0.1}
watchlist = [(dtest, 'eval'), (dtrain, 'train')]

In [11]:
bst = xgb.train(param, dtrain, 60, watchlist)

[0]	eval-error:0.027074	train-error:0.017978
[1]	eval-error:0.019214	train-error:0.017228
[2]	eval-error:0.024454	train-error:0.017228
[3]	eval-error:0.024454	train-error:0.017978
[4]	eval-error:0.022707	train-error:0.017603
[5]	eval-error:0.018341	train-error:0.016479
[6]	eval-error:0.018341	train-error:0.017228
[7]	eval-error:0.018341	train-error:0.016854
[8]	eval-error:0.018341	train-error:0.017603
[9]	eval-error:0.018341	train-error:0.017603
[10]	eval-error:0.018341	train-error:0.017603
[11]	eval-error:0.017467	train-error:0.017228
[12]	eval-error:0.017467	train-error:0.017228
[13]	eval-error:0.017467	train-error:0.017603
[14]	eval-error:0.017467	train-error:0.017603
[15]	eval-error:0.017467	train-error:0.017603
[16]	eval-error:0.017467	train-error:0.017978
[17]	eval-error:0.017467	train-error:0.017603
[18]	eval-error:0.017467	train-error:0.017228
[19]	eval-error:0.017467	train-error:0.017603
[20]	eval-error:0.017467	train-error:0.017603
[21]	eval-error:0.017467	train-error:0.01760

In [15]:
h5_files = glob.glob('../data/h5/*.h5')
dfs = [pd.read_hdf(path) for path in h5_files]

In [16]:
dfs = pd.concat(dfs)

In [20]:
df = df.rename(columns={'freq_peak.1': 'freq_peak'})

In [23]:
dfs = dfs.rename(columns={'freq_peak.1': 'freq_peak'})
dfs = dfs[nice_features]

In [26]:
p = xgb.DMatrix(dfs)

In [27]:
pred = bst.predict(p)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51'] ['freq_Q25', 'freq_Q75', 'freq_mean', 'freq_median', 'freq_mode', 'freq_peak', 'offset', 'onset', 'pitch_IQR', 'pitch_Q25', 'pitch_Q75', 'pitch_max', 'pitch_mean', 'pitch_median', 'pitch_min', 'yaafe_LPC', 'yaafe_LSF.0', 'yaafe_LSF.1', 'yaafe_LSF.2', 'yaafe_LSF.3', 'yaafe_LSF.4', 'yaafe_LSF.5', 'yaafe_LSF.6', 'yaafe_LSF.7', 'yaafe_LSF.8', 'yaafe_LSF.9', 'yaafe_MFCC.0', 'yaafe_MFCC.1', 'yaafe_MFCC.10', 'yaafe_MFCC.11', 'yaafe_MFCC.12', 'yaafe_MFCC.2', 'yaafe_MFCC.3', 'yaafe_MFCC.4', 'yaafe_MFCC.5', 'yaafe_MFCC.6', 'yaafe_MFCC.7', 'yaafe_MFCC.8', 'yaafe_MFCC.9', 'yaafe_OBSI.0', 'yaafe_OBSI.1', 'yaafe_OBS